Tartalom:

1. lexikai gyakorisagok 
2. lemmatizalas
3. lexikai diverzitas

In [1]:
# ! pip install https://github.com/oroszgy/spacy-hungarian-models/releases/download/hu_core_ud_lg-0.3.1/hu_core_ud_lg-0.3.1-py3-none-any.whl  
# kb 50 percig fut

In [2]:
import pandas as pd
import re
import hu_core_ud_lg
import nltk
import spacy
from spacy.lemmatizer import Lemmatizer
import hu_core_ud_lg
from lexical_diversity import lex_div as ld

In [3]:
df = pd.read_csv('2020-08-30_reply.csv',  sep = ";", decimal = ",", encoding = "ISO-8859-2", skipinitialspace=True)

df["reply"].replace({'ĺ\x90': 'Ő'}, inplace=True, regex=True)
df["reply"].replace({'í\xad': 'í'}, inplace=True, regex=True)

### 1. gyakorisagok szamolasa

In [4]:
szamok = []
mondatokszama = []
szavakszama = []
atlagosmondathossz = []
pont = []
vesszo = []
irj = []
felk = []
emoji = []

for text in df.reply:
    
    szamok.append(len(re.findall('[0-9]', text)))  # szamok karakterhossza
    
    sent_fre = 0
    for sent in nltk.sent_tokenize(text): # mondatok szama
        sent_fre += 1
    mondatokszama.append(sent_fre)
    
    szavakszama.append(len(re.findall(r'\w+', text))) # szavak szama az emailben    
  
    pont.append(len(re.findall('\.', text))) # pontok szama
    
    vesszo.append(len(re.findall(',', text)))  # vesszok szama
    
    felk.append(len(re.findall('!', text)))  # felkialtojelek szama
    
    irj.append(len(re.findall('[^.\w,]', re.sub(' ', '', text)))) # irasjelek szama (nem ., vagy szokoz)
    
    emoji.append(len(re.findall(':\)|:-\)|;-\)|;\)|:-D|:D|;-D|;D|:-\(', text)) ) # ezt ":(" kivettem, mert zarojelnel szerepelt
    # az emjoi python fv-t nem sikerult beuzemeltetni, mert unicode kell neki.

df["szam_char"] = szamok
df["mondatok"] = mondatokszama
df["szavak"] = szavakszama
df["mondatok_avg"] = df["mondatok"] / df["szavak"] 
df["pontok"] = pont
df["vesszok"] = vesszo
df["felk"] = felk
df["irj"] = irj
df["emoji"] = emoji

### 2. lemmatizalas

In [5]:
nlp = hu_core_ud_lg.load()
# 2-3 perc

In [8]:
output_lemma = []

for i in df.reply:
    mondat = ""
    doc = nlp(i)
    newtext = [(tok.lemma_, tok.is_title) for tok in doc]
    mondat = ' '.join([tok[0].title() if tok[1] == 1 else tok[0] for tok in newtext])
    output_lemma.append(mondat)

df.loc[:,"lemma"] = output_lemma

In [10]:
# kell a whitestrip a diverzitas stathoz
lemma_wt_whitespace = []

for text in df.lemma:
    text = text.lstrip()
    # remove whitespace after punctuation and before/after parentheses
    lemma_wt_whitespace.append(re.sub(r'(\s([,.:;!?"]))|(?<=\[|\()(.*?)(?=\)|\])', lambda x: x.group().strip(), text))

df["lemma"] = lemma_wt_whitespace 

### 3. lexikai diverzitas

In [16]:
tok = []
basic = []
simple_ttr = []
root_ttr = []
log_ttr = []

for text in df.lemma:
    tok = ld.tokenize(text)
    basic.append(len(tok) / len(set(tok)) ) # ez a simple_ttr reciprokja, azert jo mert mas iranyba nyulik el az eloszlas? 
    simple_ttr.append(ld.ttr(tok)) # unique / total
    root_ttr.append(ld.root_ttr(tok)) # unique / sqrt(total)
    log_ttr.append(ld.log_ttr(tok)) # ln(unique) / ln(total)

df["basic"] = basic
df["simple_ttr"] = simple_ttr
df["root_ttr"] = root_ttr
df["log_ttr"] = log_ttr

In [14]:
df.to_csv("2020-09-02_lemmatized.csv", sep = ";", encoding = "ISO-8859-2", header = True, index = False)